### Download and make the dataset ready in Kaggle 


In [2]:

# ## uncomment if The zip file of the dataset isn't downloaded,extraced 
# !pip install gdown
# Copy the link. The file ID is the long string of characters between d/ and /view.
#For example, in the URL https://drive.google.com/file/d/1aBcDeFgHiJkLmNoPqRsTuVwXyZ/view?usp=sharing, 
#the file ID is 1aBcDeFgHiJkLmNoPqRsTuVwXyZ
# !mkdir /kaggle/tmp
# !gdown  1pzXpA5Cz0DJmjRsLxlqRNnJq-kOUvojb -O /kaggle/tmp/Labeled_CICMODBUS2023.zip
# !unzip /kaggle/tmp/Labeled_CICMODBUS2023.zip -d /kaggle/working/

# # ## uncomment if the python modules (modbus.py,utils.py ,...) not cloned  and added to the path 

# !git clone https://github.com/hamid-rd/FLBased-ICS-NIDS.git
# import sys
# # Add the repository folder to the Python path
# repo_path = '/kaggle/working/FLBased-ICS-NIDS'
# repo_input_path = '/kaggle/input/training/FLBased-ICS-NIDS'
# dataset_path = '/kaggle/input/training/'

# for path in {repo_path,repo_input_path,dataset_path}:
#     if path not in sys.path:
#         sys.path.append(path)


In [3]:
# To test if every thing is okay (modbus.py class and correct number of founded csv files )
from modbus import ModbusDataset,ModbusFlowStream

# dataset_directory = "/kaggle/working/ModbusDataset" 
# dataset_directory = "/kaggle/input/training/ModbusDataset" 
dataset_directory = "dataset" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

# Don't forget to save version in kaggle (to save outputs written on the disk (/kaggle/working/))  

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

### Unsupervised Autoencoder training  

In [4]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np # For standard deviation calculation
from modbus import ModbusDataset,ModbusFlowStream
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,recall_score
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from utils import load_scalers
import random
from random import SystemRandom
from sklearn.model_selection import train_test_split
import itertools
import torch.nn.init as init
from collections import Counter

def compute_threshold(mse_values,k=1):

    """
    K-SIGMA
    Computes the anomaly detection threshold (for marking sample as Intrusion if the IS was greater )
    based on the mean and standard deviation of Mean Squared Error (MSE) values.
    Formula: thr = mean(MSE) + std(MSE)
    Args:
    mse_values (torch.Tensor or list/np.array): A tensor or list of MSE values

                            obtained from the validation set.
    Returns:
    float: The calculated threshold.
    float: The calculated std.

    """
    if not isinstance(mse_values, torch.Tensor):
        mse_values = torch.tensor(mse_values, dtype=torch.float32)
    if mse_values.numel() == 0:
        return 0.0
    mean_mse = torch.mean(mse_values)
    std_mse = torch.std(mse_values)
    print("-----------mse_loss mean : ",f"{mean_mse.item():.4f}","std:",f"{std_mse.item():.4f}")
    threshold = mean_mse + k*std_mse
    return threshold.item(),std_mse.item()

def vae_loss_function(recon_x, x, mu, logvar,beta=0.05):
    """
    VAE loss function.
    """
    #equivalent to BCE loss
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # print(BCE.item(),KLD.item())
    return (BCE + beta*KLD)

def _init_weights( module):
    ## for one layer apply Xavier Initialization
    if isinstance(module, nn.Linear):
        init.xavier_normal_(module.weight)
        if module.bias is not None:
            init.zeros_(module.bias)
    return module


In [38]:
# dataset_directory = "/kaggle/input/training/ModbusDataset" # change this to the folder contain benign and attack subdirs
# dataset_directory = "dataset" 
dataset_directory = "./ModbusDataset/" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

In [39]:

# AutoEncoder (AE)
class AE(nn.Module):
    """
    Encoder: (76-32-16-4-2)
    Decoder: (2-4-16-32-76)
    """
    def __init__(self,input_dim=76):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon


# Variational AutoEncoder (VAE)
class VAE(nn.Module):
    """
    Encoder: (76-16-4-2 for mu and log_var)
    Decoder: (2-4-16-76)
    return x_recon, mu, logvar
    """
    def __init__(self,input_dim=76):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(4, 2)
        self.fc_logvar = nn.Linear(4, 2)
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, input_dim),
            nn.Sigmoid()
                    )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar

    
class AAE_Encoder(nn.Module):
    def __init__(self,input_dim=76):
        """
        Encoder(Generator):(76-16-4-2)
        """
        super(AAE_Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 4),
            nn.LeakyReLU(0.2),
            nn.Linear(4, 2))
    def forward(self, x):
        return self.encoder(x)
class AAE_Decoder(nn.Module):
    def __init__(self,input_dim=76):
        super(AAE_Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 16),
            nn.LeakyReLU(),
            nn.Linear(16, input_dim),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.decoder(x)
class AAE_Discriminator(nn.Module):
    def __init__(self):
        super(AAE_Discriminator, self).__init__()
        # corrected to 2-16-4-1
        self.discriminator = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 1), 
            nn.Sigmoid()
        )    
    def forward(self, x):
        return self.discriminator(x)
 
class AdversarialAutoencoder(nn.Module):
    def __init__(self):
        super(AdversarialAutoencoder, self).__init__()
        self.encoder = AAE_Encoder()
        self.decoder = AAE_Decoder()
        self.discriminator = AAE_Discriminator()
    def forward(self, x):
        fake_z = self.encoder(x)
        x_recon = self.decoder(fake_z)
        return fake_z,x_recon


In [7]:
from torch.utils.data import Dataset
class InMemoryDataset(Dataset):
    """A simple dataset that serves data from pre-loaded tensors."""
    def __init__(self, data, labels):
        # This dataset holds references to the data, which should already be in shared memory.
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

### Part a: Centralized learning 

##### You can go from here right to the FL part


In [40]:
def make_dataloader(load_all_chunks=True):

    ## must be called after that the global train_files,val_files and test_files plus loades scalers are initilized
    ## return train, valid and test dataloaders
    if load_all_chunks==True:
        large_chunk_size = modbus.dataset["metadata"]["founded_files_num"]["total_dataset_num"]

        dataset_configs = {
            "train": {"files": train_files},
            "val": {"files": val_files},
            "test": {"files": test_files},
        }
        datasets = {}
        ae_datasets = {}
        dataloaders = {}

        print("Cow Processing datasets...")

        for name, config in dataset_configs.items():
            print(f"  - Creating '{name}' dataset...")
            
            # 1. Create the primary ModbusFlowStream dataset
            datasets[name] = ModbusFlowStream(
                shuffle=False,
                chunk_size=large_chunk_size,
                batch_size=1,
                csv_files=config["files"],
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
                window_size=window_size
            )
            
            # 2. Call __getitem__(0) once to load the entire dataset chunk into memory
            datasets[name].__getitem__(0)

            # 2. Extract the raw tensors
            all_data = datasets[name].current_chunk_data
            all_labels = datasets[name].current_chunk_labels
            
            # all_data.share_memory_()
            # all_labels.share_memory_()
            
            # 4. Create an instance of our SIMPLE dataset using the SHARED tensors.
            shared_dataset = InMemoryDataset(all_data, all_labels)

            # 5. Create the DataLoader from the simple dataset. This will work correctly with workers.
            if name=="train":
                shuffle_samples =True
            else:
                shuffle_samples =False
            # dataloaders[name] = DataLoader(
            #     shared_dataset,
            #     batch_size=64,
            #     shuffle=shuffle_samples,
            #     num_workers=4
            #     persistent_workers=True,
            #     pin_memory=True
            # )
            dataloaders[name] = DataLoader(
                shared_dataset,
                batch_size=64,
                shuffle=shuffle_samples,
                num_workers=0, # You can now use multiple workers effectively.
                persistent_workers=False,
                pin_memory=False
            )
                    
            
            print(f"  - Finished '{name}' dataset.")
        train_dataloader = dataloaders['train']
        val_dataloader = dataloaders['val']
        test_dataloader = dataloaders['test']

    else :
        train_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=True,chunk_size=1,batch_size=64,csv_files=train_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),  batch_size=1,shuffle=False)
        val_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),batch_size=1,shuffle=False)
        test_dataloader=DataLoader(ModbusFlowStream(shuffle=False,chunk_size=1,batch_size=64,csv_files=test_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size),
                                    batch_size=1,shuffle=False)
    return train_dataloader,val_dataloader,test_dataloader


In [72]:

def train_eval(model,train_dataloader,val_dataloader,test_dataloader,learning_rates= [5e-6,1e-7,5e-5,1e-5,1e-6],
               weight_decays=[1e-5,1e-4,1e-7],shuffle_files=True,num_epochs=20,eval_epoch=4,criterion_method="mse", k_range=[1,3],train_model=True):
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model=model.to(device)
    if criterion_method=="bce":
        criterion = nn.BCELoss(reduction='sum').to(device)
        eval_criterion = nn.BCELoss(reduction='none').to(device)
    else: #mse
        criterion = nn.MSELoss(reduction='sum').to(device)
        eval_criterion = nn.MSELoss(reduction='none').to(device)
    best_f1=0 #to save best version of the model during test
    best_recall=0 #to save best version of the model during test

    for lr, wd in itertools.product(learning_rates, weight_decays):
        if model._get_name()=="AdversarialAutoencoder":
            adversarial_criterion= nn.BCELoss(reduction="sum")
            optimizer_D = optim.Adam(model.discriminator.parameters(), lr=lr, weight_decay=wd)
            optimizer_G =  optim.Adam(list(model.encoder.parameters()) + list(model.decoder.parameters()), lr=lr, weight_decay=wd)
        else:
            AE_optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
            ### new code
            # AE_optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd)

        print(f"\n==================  lr={lr}, wd={wd} ==================")
        if train_model==True:
            model.apply(_init_weights)
        for epoch in range(num_epochs):
            if train_model==True:
                time_1 = time.time()
                model.train()
                train_loss = 0
                ## for AAE
                Discriminator_loss = 0
                for sequences, labels in train_dataloader:
                    sequences=sequences.squeeze().to(device)
                    if labels.sum()!=0:
                        continue
                    if model._get_name()=="AdversarialAutoencoder":
                        # 1) generator loss

                        target_ones= torch.ones(sequences.size(0), 1,device=device,dtype=torch.float)
                        target_zeros= torch.zeros(sequences.size(0), 1,device=device,dtype=torch.float)
                        random_latent = torch.randn(sequences.size(0), 2, device=device)
                        optimizer_G.zero_grad()
                        fake_z,decoded_seq = model(sequences)
                        G_loss = 0.001*adversarial_criterion(model.discriminator(fake_z),target_ones ) + 0.999*criterion(decoded_seq, sequences)
                        G_loss.backward()
                        optimizer_G.step()
                        # 2) discriminator loss
                        optimizer_D.zero_grad()
                        real_loss = adversarial_criterion(model.discriminator(random_latent), target_ones)
                        fake_loss = adversarial_criterion(model.discriminator(fake_z.detach()),  target_zeros)
                        D_loss = 0.5*(real_loss + fake_loss)
                        D_loss.backward()
                        optimizer_D.step()
                        train_loss+=G_loss.item()
                        Discriminator_loss+=D_loss.item()   
                    else:
                        AE_optimizer.zero_grad()
                        if model._get_name()=="AE":
                            recon = model(sequences)
                            loss = criterion(recon, sequences) / sequences.size(0)
                        elif model._get_name()=="VAE" :
                            recon, mu, logvar = model(sequences)
                            loss = vae_loss_function(recon, sequences, mu, logvar) /sequences.size(0)
                        loss.backward()
                        AE_optimizer.step()
                        train_loss += loss.item()
                print(f"Train : time {(time.time()-time_1):.2f} s",
                f"Epoch {epoch+1}")
                if model._get_name()=="AdversarialAutoencoder":
                    print(f"Generator Loss: {train_loss / len(train_dataloader):.4f}",
                        f"Discriminator Loss: {Discriminator_loss / len(train_dataloader):.4f}")
                else:
                    print(f"Train Loss: {train_loss / len(train_dataloader):.4f}")
            # Evaluate part
            if (epoch + 1) % eval_epoch == 0:
                model.eval() 
                all_val_losses = []
                all_val_labels = []
                print(f"--- Running Evaluation for Epoch {epoch+1} lr ={lr} wd {wd} ---")
                with torch.no_grad():
                    for sequences, labels in val_dataloader:
                        sequences = sequences.squeeze().to(device) 
                        if criterion_method=="bce":
                            ## may test features be greater than 1 after scaling 
                            sequences=torch.clamp(sequences, min=0.0, max=1.0)      
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE" :
                            recon, _, _ = model(sequences)
                        elif model._get_name()=="AdversarialAutoencoder":
                            _,recon= model(sequences)
                        val_loss = eval_criterion(recon, sequences)
                        if val_loss.dim() > 1:
                            val_loss = val_loss
                        else:
                            val_loss = val_loss.unsqueeze(dim=0)
                            labels = labels.unsqueeze(dim=0)
                        val_loss = val_loss.sum(dim=1)
                        all_val_losses.extend(val_loss.cpu().numpy())
                        all_val_labels.extend(labels.flatten().cpu().numpy())
                threshold_1,std_mse = compute_threshold(all_val_losses,k=0)

                all_val_losses = np.array(all_val_losses).squeeze()  
                all_val_labels = np.array(all_val_labels).squeeze()  
                # If intrusion score > threshold, predict 1 (intrusion), else 0 (benign)
                # For FDR, get True Positives (TP) and False Positives (FP)
                
                predictions = (all_val_losses > threshold_1).astype(int)

                accuracy = accuracy_score(all_val_labels, predictions)
                print(f"Val: Accuracy: {accuracy:.4f}  ")
                model.eval() 
                all_test_losses = []
                all_test_labels = []
                with torch.no_grad():
                    for sequences, labels in test_dataloader:
                        sequences = sequences.squeeze().to(device)
                        labels = labels.squeeze().to(device)
                        if criterion_method=="bce":
                            ## may test features be greater than 1 after scaling 
                            sequences=torch.clamp(sequences, min=0.0, max=1.0)
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE"  :
                            recon, mu, logvar = model(sequences)
                        elif model._get_name()=="AdversarialAutoencoder":
                            _,recon= model(sequences)

                        intrusion_scores = eval_criterion(recon, sequences)
                        if intrusion_scores.dim() > 1:
                            intrusion_scores = intrusion_scores
                        else:
                            intrusion_scores = intrusion_scores.unsqueeze(dim=0)
                            labels = labels.unsqueeze(dim=0)
                        if intrusion_scores.dim()==3:
                            ##GRU : mean of window
                            intrusion_scores = intrusion_scores.mean(dim=1)
                        intrusion_scores = intrusion_scores.sum(dim=1)
                        all_test_losses.extend(intrusion_scores.cpu().numpy())
                        all_test_labels.extend(labels.cpu().numpy())

                all_test_losses = np.array(all_test_losses)
                all_test_labels = np.array(all_test_labels)
                temp_best_recall =best_recall
                temp_best_f1 =best_f1

                for k in k_range:
                    threshold=threshold_1+k*std_mse
                    print(f" K: {k} K-SIGMA Threshold : ---thr {threshold:.4}")
                    predictions = (all_test_losses > threshold).astype(int)
                    binary_test_labels = (all_test_labels != 0).astype(int)

                    # Find the indices where the prediction was incorrect
                    misclassified_indices = np.where(binary_test_labels != predictions)[0]

                    # Get the original labels for those misclassified instances
                    misclassified_original_labels = all_test_labels[misclassified_indices]

                    # To get a summary count of which labels were misclassified
                    print("Counts of : original binary labels",Counter(binary_test_labels),"predicted binary labels",Counter(predictions))
                    print(f"Counts of  original  labels: {dict(sorted(Counter(all_test_labels).items()))}")
                    print(f"Counts of misclassified original labels: {dict(sorted(Counter(misclassified_original_labels).items()))}")
                    accuracy = accuracy_score(binary_test_labels, predictions)
                    f1 = f1_score(binary_test_labels, predictions, zero_division=0)
                    recall = recall_score(binary_test_labels, predictions,zero_division=0)
                    _, fp, _, tp = confusion_matrix(binary_test_labels, predictions, labels=[0, 1]).ravel()
                    # FDR = FP / (FP + TP) 
                    if (fp + tp) == 0:
                        fdr = 0.0 
                    else:
                        fdr = fp / (fp + tp)
                    print(f"Test : Accuracy: {accuracy:.4f} Recall : {recall:.4f} FDR: {fdr:.4f}  F1-score: {f1:.4f}  ")
                    !mkdir best_models -p
                    if f1>best_f1 :
                        best_f1=f1
                    if recall>best_recall:
                        best_recall=recall
                if (best_recall>temp_best_recall or best_f1 > temp_best_f1):
                    if train_model==True:
                        save_path ="best_models/"+model._get_name()+"_f1_"+f"{best_f1:.2f}" +"_recall_"+f"{best_recall:.2f}" +"_.pth"
                        torch.save(model.state_dict(),save_path)
                        print("model",model._get_name(),"is saved in" ,save_path )


#### Train on network-wdie
#### evaluate on comrpomised-scada IED node ied1b during centralized training.

In [74]:
train_files=[col for col in modbus.dataset["benign_dataset_dir"] if col.find("network-wide")!=-1][:]
test_files= [col for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("ied1b")!=-1]


### missed attack logs files for the day 21 for ied1b which can reduce the accuracy.
test_files.remove(dataset_directory+"/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")
random.seed(42)
random.shuffle(train_files)
random.shuffle(test_files)
val_files = train_files[-4:]
train_files = train_files[:-4]
print("ied1b comp ied attack ->\n test: ",len(test_files),test_files)
print("----------- Network-wide number of csv files -> \n ----------- train :",len(train_files),train_files,"\n -------- valid:",len(val_files),val_files)

ied1b comp ied attack ->
 test:  7 ['./ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-1-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-5-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-3-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-7-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-2-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-4-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-0-labeled.csv']
----------- Network-wide number of csv files -> 
 ----------- train : 15 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-16-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture

In [75]:

window_size=1
torch.manual_seed(42)
np.random.seed(42)
loaded_scalers=load_scalers("fitted_scalers")

Successfully loaded scalers for 'network-wide'


In [76]:
def make_dataloader(load_all_chunks=True):

    ## must be called if global train_files,val_files and test_files plus loades scalers are initilized
    ## return train, valid and test dataloaders
    if load_all_chunks:
        large_chunk_size = modbus.dataset["metadata"]["founded_files_num"]["total_dataset_num"]

        dataset_configs = {
            "train": {"files": train_files},
            "val": {"files": val_files},
            "test": {"files": test_files},
        }
        datasets = {}
        ae_datasets = {}
        dataloaders = {}

        print("Cow Processing datasets...")

        for name, config in dataset_configs.items():
            print(f"  - Creating '{name}' dataset...")
            
            # 1. Create the primary ModbusFlowStream dataset
            datasets[name] = ModbusFlowStream(
                shuffle=False,
                chunk_size=large_chunk_size,
                batch_size=1,
                csv_files=config["files"],
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
                window_size=window_size
            )
            
            # 2. Call __getitem__(0) once to load the entire dataset chunk into memory
            datasets[name].__getitem__(0)

            # 2. Extract the raw tensors
            all_data = datasets[name].current_chunk_data
            all_labels = datasets[name].current_chunk_labels
            
            # all_data.share_memory_()
            # all_labels.share_memory_()
            
            # 4. Create an instance of our SIMPLE dataset using the SHARED tensors.
            shared_dataset = InMemoryDataset(all_data, all_labels)

            # 5. Create the DataLoader from the simple dataset. This will work correctly with workers.
            if name=="train":
                shuffle_samples =True
            else:
                shuffle_samples =False
            # dataloaders[name] = DataLoader(
            #     shared_dataset,
            #     batch_size=64,
            #     shuffle=shuffle_samples,
            #     num_workers=4
            #     persistent_workers=True,
            #     pin_memory=True
            # )
            dataloaders[name] = DataLoader(
                shared_dataset,
                batch_size=64,
                shuffle=shuffle_samples,
                num_workers=0, # You can now use multiple workers effectively.
                persistent_workers=False,
                pin_memory=False
            )
                    
            
            print(f"  - Finished '{name}' dataset.")
        train_dataloader = dataloaders['train']
        val_dataloader = dataloaders['val']
        test_dataloader = dataloaders['test']

    else :
        train_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=True,chunk_size=1,batch_size=64,csv_files=train_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),  batch_size=1,shuffle=False)
        val_dataloader=DataLoader(ModbusFlowStream( 
            shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size
        ),batch_size=1,shuffle=False)
        test_dataloader=DataLoader(ModbusFlowStream(shuffle=False,chunk_size=1,batch_size=64,csv_files=test_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=window_size),
                                    batch_size=1,shuffle=False)
    return train_dataloader,val_dataloader,test_dataloader


In [78]:
train_dataloader,val_dataloader,test_dataloader= make_dataloader(load_all_chunks=False)


In [ ]:
t_1=time.time()
for seq,label in train_dataloader:
     pass
print(time.time()-t_1)

20.64165949821472


In [79]:
print(len(train_dataloader),len(val_dataloader),len(test_dataloader))

34527 9584 11720


In [ ]:
AE_model = AE(input_dim=76)
train_eval(AE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=6,eval_epoch=2,criterion_method="mse",learning_rates=[5e-3,1e-3,5e-4,1e-4],weight_decays=[1e-5,1e-6],k_range=[1,3])
# --- Running Evaluation for Epoch 6 lr =0.001 wd 1e-05 ---



==================  lr=0.005, wd=1e-05 ==================
Train : time 156.91 s Epoch 1
Train Loss: 0.1165
Train : time 154.89 s Epoch 2
Train Loss: 0.0165
--- Running Evaluation for Epoch 2 lr =0.005 wd 1e-05 ---
-----------mse_loss mean :  0.0154 std: 0.2341
Val: Accuracy: 0.9908  
 K: 1 K-SIGMA Threshold : ---thr 0.2495
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 561817, 1: 187962})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misclassified original labels: {0: 7941, 1: 40034, 2: 58, 3: 96, 4: 42, 5: 2, 6: 68, 7: 1}
Test : Accuracy: 0.9357 Recall : 0.8171 FDR: 0.0422  F1-score: 0.8818  
 K: 3 K-SIGMA Threshold : ---thr 0.7176
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 731959, 1: 17820})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of miscla

In [82]:
AE_model = AE(input_dim=76)
train_eval(AE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=15,eval_epoch=5,criterion_method="mse",learning_rates=[5e-4],weight_decays=[1e-5],k_range=[1])



==================  lr=0.0005, wd=1e-05 ==================
Train : time 165.37 s Epoch 1
Train Loss: 0.1599
Train : time 167.19 s Epoch 2
Train Loss: 0.0203
Train : time 164.12 s Epoch 3
Train Loss: 0.0105
Train : time 160.85 s Epoch 4
Train Loss: 0.0103
Train : time 141.08 s Epoch 5
Train Loss: 0.0100
--- Running Evaluation for Epoch 5 lr =0.0005 wd 1e-05 ---
-----------mse_loss mean :  0.0086 std: 0.1457
Val: Accuracy: 0.9933  
 K: 1 K-SIGMA Threshold : ---thr 0.1543
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 564510, 1: 185269})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misclassified original labels: {0: 7898, 1: 42457, 2: 59, 3: 99, 4: 32, 5: 2, 6: 70, 7: 232}
Test : Accuracy: 0.9322 Recall : 0.8051 FDR: 0.0426  F1-score: 0.8746  
model AE is saved in best_models/AE_f1_0.87_recall_0.81_.pth
Train : time 152.46 s Epoch 6
Train Loss: 0.0097
Train : time 154.

In [ ]:
VAE_model = VAE(input_dim=76)
train_eval(VAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=6,eval_epoch=2,criterion_method="mse",learning_rates=[1e-3,1e-4],weight_decays=[1e-5,1e-6],k_range=[1,3])



==================  lr=0.001, wd=1e-05 ==================
Train : time 160.35 s Epoch 1
Train Loss: 0.2666
Train : time 154.92 s Epoch 2
Train Loss: 0.1515
--- Running Evaluation for Epoch 2 lr =0.001 wd 1e-05 ---
-----------mse_loss mean :  0.0407 std: 0.2427
Val: Accuracy: 0.8383  
 K: 1 K-SIGMA Threshold : ---thr 0.2834
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 557414, 1: 192365})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misclassified original labels: {0: 11974, 1: 39666, 2: 58, 3: 94, 4: 41, 5: 2, 6: 67, 7: 3}
Test : Accuracy: 0.9308 Recall : 0.8188 FDR: 0.0622  F1-score: 0.8742  
 K: 3 K-SIGMA Threshold : ---thr 0.7688
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 562932, 1: 186847})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misc

KeyboardInterrupt: 

In [83]:
VAE_model = VAE(input_dim=76)
train_eval(VAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=10,eval_epoch=5,criterion_method="mse",learning_rates=[1e-3],weight_decays=[1e-5,],k_range=[1])



==================  lr=0.001, wd=1e-05 ==================
Train : time 165.04 s Epoch 1
Train Loss: 0.3184
Train : time 155.17 s Epoch 2
Train Loss: 0.1613
Train : time 158.38 s Epoch 3
Train Loss: 0.1547
Train : time 168.95 s Epoch 4
Train Loss: 0.1528
Train : time 163.59 s Epoch 5
Train Loss: 0.1508
--- Running Evaluation for Epoch 5 lr =0.001 wd 1e-05 ---
-----------mse_loss mean :  0.0367 std: 0.2408
Val: Accuracy: 0.8536  
 K: 1 K-SIGMA Threshold : ---thr 0.2775
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 555983, 1: 193796})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misclassified original labels: {0: 13336, 1: 39597, 2: 57, 3: 95, 4: 42, 5: 2, 6: 68, 7: 1}
Test : Accuracy: 0.9290 Recall : 0.8191 FDR: 0.0688  F1-score: 0.8715  
model VAE is saved in best_models/VAE_f1_0.87_recall_0.82_.pth
Train : time 168.75 s Epoch 6
Train Loss: 0.1486
Train : time 159.5

In [ ]:
AAE_model = AdversarialAutoencoder()
train_eval(AAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=6,eval_epoch=3,criterion_method="mse",learning_rates=[1e-3,1e-4],weight_decays=[1e-5,1e-6])



==================  lr=0.001, wd=1e-05 ==================
Train : time 231.25 s Epoch 1
Generator Loss: 7.7712 Discriminator Loss: 13.8002
Train : time 239.30 s Epoch 2
Generator Loss: 1.1697 Discriminator Loss: 18.9262
Train : time 253.08 s Epoch 3
Generator Loss: 1.1140 Discriminator Loss: 22.2141
--- Running Evaluation for Epoch 3 lr =0.001 wd 1e-05 ---
-----------mse_loss mean :  0.0155 std: 0.2409
Val: Accuracy: 0.9942  
 K: 1 K-SIGMA Threshold : ---thr 0.2564
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 568636, 1: 181143})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misclassified original labels: {0: 7874, 1: 46780, 2: 61, 3: 99, 4: 40, 5: 2, 6: 70, 7: 1}
Test : Accuracy: 0.9267 Recall : 0.7864 FDR: 0.0435  F1-score: 0.8632  
 K: 3 K-SIGMA Threshold : ---thr 0.7381
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Co

In [85]:
AAE_model = AdversarialAutoencoder()
train_eval(AAE_model,train_dataloader,val_dataloader,test_dataloader,shuffle_files=True,num_epochs=30,eval_epoch=5,criterion_method="mse",learning_rates=[1e-4],weight_decays=[1e-6])



==================  lr=0.0001, wd=1e-06 ==================
Train : time 266.33 s Epoch 1
Generator Loss: 56.0338 Discriminator Loss: 6.7792
Train : time 264.11 s Epoch 2
Generator Loss: 9.7689 Discriminator Loss: 12.3131
Train : time 263.87 s Epoch 3
Generator Loss: 8.1489 Discriminator Loss: 10.4486
Train : time 261.28 s Epoch 4
Generator Loss: 6.0716 Discriminator Loss: 11.2788
Train : time 256.96 s Epoch 5
Generator Loss: 5.6057 Discriminator Loss: 10.5377
--- Running Evaluation for Epoch 5 lr =0.0001 wd 1e-06 ---
-----------mse_loss mean :  0.0837 std: 0.3804
Val: Accuracy: 0.9290  
 K: 1 K-SIGMA Threshold : ---thr 0.4641
Counts of : original binary labels Counter({0: 529457, 1: 220322}) predicted binary labels Counter({0: 561825, 1: 187954})
Counts of  original  labels: {0: 529457, 1: 218884, 2: 317, 3: 236, 4: 243, 5: 240, 6: 170, 7: 232}
Counts of misclassified original labels: {0: 7937, 1: 40041, 2: 58, 3: 96, 4: 39, 5: 2, 6: 68, 7: 1}
Test : Accuracy: 0.9357 Recall : 0.8171 F

KeyboardInterrupt: 

#### Evaluate pre-trained autoencoders  on the compromised-ied and compromised scada scenarios 

No exact labeling for the comp ied scenario results in low performance 

In [ ]:
Trained_AE_model=AE(input_dim=76)
Trained_AE_model.load_state_dict(torch.load("./best_models/AE_f1_0.88_recall_0.81_.pth"))
Trained_VAE_model=VAE(input_dim=76)
Trained_VAE_model.load_state_dict(torch.load("./best_models/VAE_f1_0.88_recall_0.82_.pth"))
Trained_AAE_model=AdversarialAutoencoder()
Trained_AAE_model.load_state_dict(torch.load("./best_models/AdversarialAutoencoder_f1_0.88_recall_0.82_.pth"))

<All keys matched successfully>

In [66]:
dataset_directory = "./ModbusDataset" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()


 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

In [68]:
for scenario in {"compromised-ied","external","compromised-scada"}:
    if scenario=="compromised-scada":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("ied1b")!=-1]
        ### missed attack logs for the day 21 for ied1b which can reduce the accuracy.
        test_files.remove(dataset_directory+"/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")    
    elif scenario=="compromised-ied":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("trust-scada-hmi")!=-1]
    else:
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("network-wide")!=-1]        

    print("----------- benign valid files:",len(val_files),val_files)
    print(f"----------{scenario} attack  test files : ",len(test_files),test_files)
    val_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=val_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    test_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=test_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
        print("*"*10,trained_model._get_name(),10*"*")
        train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
        

scenario : compromised-scada
----------- benign valid files: 4 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-24-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-28-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-30-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-22-labeled.csv']
----------compromised-scada attack  test files :  7 ['./ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-3-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-4-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-1-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-7-labeled.csv', './ModbusDataset/attack/compromised-sca

In [52]:

## TEST RESULTS WITHOUT IED1B TRAFFIC
modbus = ModbusDataset(dataset_directory,"ready")

for scenario in {"external","compromised-scada"}:

    print(scenario)
    test_files=modbus.dataset["attack_dataset_dir"][scenario]
    nodes_to_exclude = [
    "ied1b",
    "network-wide",
    "substation-wide-capture",
    "scada-hmi-network-capture",
    "trust-scada-hmi",
    "central-agent",
    "ied4c"
    ]
    exclude = [col for col in test_files if any(node in col for node in nodes_to_exclude)]
    [test_files.remove(c) for c in exclude if c in test_files]
    print("exclude",len(exclude),exclude)
    print("filtered test files without ied1b",len(test_files),test_files)
    for test_file in test_files: 
        print(test_file)
        val_dataloader=DataLoader(ModbusFlowStream(
                    shuffle=False,
                    chunk_size=1,
                    batch_size=64,
                    csv_files=val_files,
                    scalers=loaded_scalers['network-wide']['min_max_scalers'],
                ),batch_size=1,shuffle=False)
        test_dataloader=DataLoader(ModbusFlowStream(
                    shuffle=False,
                    chunk_size=1,
                    batch_size=64,
                    csv_files=[test_file],
                    scalers=loaded_scalers['network-wide']['min_max_scalers'],
                ),batch_size=1,shuffle=False)
        for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
            print("*"*10,trained_model._get_name(),10*"*")
            train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
            

compromised-scada
exclude 51 ['./ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-0-labeled.csv', './ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-5-labeled.csv', './ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-1-labeled.csv', './ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-2-labeled.csv', './ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-4-labeled.csv', './ModbusDataset/attack/compromised-scada/ied4c/ied4c-network-captures/ready/vethe685ac9-3-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-3-labeled.csv', './ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-4-labeled.csv', './ModbusDataset/attack/c

### Part b: Federated learning 
####  non iid distribution of dataset (ip\node based)

In [7]:
# ==============================================================================
# 1. SETUP: INSTALL LIBRARIES AND IMPORT DEPENDENCIES
# ==============================================================================
# In a Kaggle notebook, run this cell first to install the necessary libraries.
# !pip install -q flwr[simulation] torch torchvision pandas scikit-learn matplotlib seaborn


In [8]:

from collections import OrderedDict
from typing import Dict, List, Tuple, Optional , Union
import os 
import flwr as fl
import ray
from flwr.common import FitRes, Scalar,Context, ndarrays_to_parameters, parameters_to_ndarrays
from flwr.server.client_proxy import ClientProxy

import random

# Suppress warning messages for a cleaner output
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
# Set a seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#global device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")
modbus = ModbusDataset(dataset_directory,"ready")


Training on cuda:0


In [9]:

# ==============================================================================
#  FEDERATED LEARNING CLIENT: FlowerClient
# ==============================================================================
class FlowerClient(fl.client.NumPyClient):
    """Flower client for training."""
    def __init__(self, cid, model, trainloader,valloader):
        self.cid = cid
        self.model = model
        self.train_dataloader = trainloader
        self.val_dataloader = valloader
    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        model =self.model
        lr = cfg.LEARNING_RATE
        wd= cfg.WEIGHT_DECAY
        
        criterion = nn.MSELoss(reduction='sum').to(DEVICE)
        if model._get_name()=="AdversarialAutoencoder":
            adversarial_criterion= nn.BCELoss(reduction="sum")
            optimizer_D = optim.Adam(model.discriminator.parameters(), lr=lr, weight_decay=wd)
            optimizer_G =  optim.Adam(list(model.encoder.parameters()) + list(model.decoder.parameters()), lr=lr, weight_decay=wd)
        else:
            AE_optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=wd)

        if cfg.STRATEGY == "FED_PROX":
            global_params_dict = {
                k: torch.tensor(v, device=DEVICE) 
                for k, v in zip(self.model.state_dict().keys(), parameters)
            }

        for epoch in range(cfg.LOCAL_EPOCHS):
            time_1 = time.time()
            model.train()
            train_loss = 0
            ## for AAE
            Discriminator_loss = 0
            for sequences, _ in self.train_dataloader:
                sequences=sequences.squeeze().to(DEVICE)
                if model._get_name()=="AdversarialAutoencoder":
                    target_ones= torch.ones(sequences.size(0), 1,device=DEVICE,dtype=torch.float)
                    target_zeros= torch.zeros(sequences.size(0), 1,device=DEVICE,dtype=torch.float)
                    random_latent = torch.randn(sequences.size(0), 2, device=DEVICE)
                    optimizer_G.zero_grad()
                    fake_z,decoded_seq = model(sequences)
                    G_loss = 0.001*adversarial_criterion(model.discriminator(fake_z),target_ones ) + 0.999*criterion(decoded_seq, sequences)
                    if cfg.STRATEGY == "FED_PROX":
                        proximal_term_G = 0.0
                        # Proximal term for ENCODER
                        for name, local_param in model.encoder.named_parameters():
                            global_param = global_params_dict['encoder.' + name]
                            proximal_term_G += torch.pow((local_param - global_param).norm(2), 2)
                        # Proximal term for DECODER
                        for name, local_param in model.decoder.named_parameters():
                            global_param = global_params_dict['decoder.' + name]
                            proximal_term_G += torch.pow((local_param - global_param).norm(2), 2)
                        
                        G_loss += (cfg.PROXIMAL_MU / 2) * proximal_term_G
                    G_loss.backward()
                    optimizer_G.step()
                    # 2) discriminator loss
                    optimizer_D.zero_grad()
                    real_loss = adversarial_criterion(model.discriminator(random_latent), target_ones)
                    fake_loss = adversarial_criterion(model.discriminator(fake_z.detach()),  target_zeros)
                    D_loss =  0.5*(real_loss + fake_loss)
                    if cfg.STRATEGY == "FED_PROX":
                        proximal_term_D = 0.0
                        # Proximal term for DISCRIMINATOR
                        for name, local_param in model.discriminator.named_parameters():
                            global_param = global_params_dict['discriminator.' + name]
                            proximal_term_D += torch.pow((local_param - global_param).norm(2), 2)
                        D_loss += (cfg.PROXIMAL_MU / 2) * proximal_term_D
            
                    D_loss.backward()
                    optimizer_D.step()
                    train_loss+=G_loss.item()
                    Discriminator_loss+=D_loss.item()   
                else:
                    AE_optimizer.zero_grad()
                    if model._get_name()=="AE":
                        recon = model(sequences)
                        loss = criterion(recon, sequences) / sequences.size(0)
                    elif model._get_name()=="VAE" :
                        recon, mu, logvar = model(sequences)
                        loss = vae_loss_function(recon, sequences, mu, logvar) /sequences.size(0)
                    
                    if cfg.STRATEGY == "FED_PROX":
                        proximal_term = 0.0
                        for name, local_param in model.encoder.named_parameters():
                            global_param = global_params_dict['encoder.' + name]
                            proximal_term += torch.pow((local_param - global_param).norm(2), 2)
                        # Proximal term for DECODER
                        for name, local_param in model.decoder.named_parameters():
                            global_param = global_params_dict['decoder.' + name]
                            proximal_term += torch.pow((local_param - global_param).norm(2), 2)
                        loss+= (cfg.PROXIMAL_MU / 2) *proximal_term
                    loss.backward()
                    AE_optimizer.step()
                    train_loss += loss.item()
            print(f"Train : time {(time.time()-time_1):.2f} s",
            f"Epoch {epoch+1}")
            num_samples=len(self.train_dataloader)
            if model._get_name()=="AdversarialAutoencoder":
                print(f"Generator Loss: {train_loss / num_samples:.4f}",
                    f"Discriminator Loss: {Discriminator_loss / num_samples:.4f}")
            else:
                print(f"Train Loss: {train_loss / num_samples:.4f}")
        local_threshold,len_val_samples=self._calculate_threshold()
        metrics = {"threshold": local_threshold,"len": len_val_samples}
        return self.get_parameters(config={}), num_samples, metrics

    def _calculate_threshold(self):
        model = self.model
        model.to(DEVICE)
        model.eval()
        eval_criterion = nn.MSELoss(reduction='none').to(DEVICE)

        # Evaluate part
        all_val_losses = []
        all_val_labels = []
        with torch.no_grad():
            for sequences, labels in self.val_dataloader:
                sequences = sequences.squeeze().to(DEVICE) 
                if model._get_name()=="AE":
                    recon = model(sequences)
                elif model._get_name()=="VAE" :
                    recon, _, _ = model(sequences)
                elif model._get_name()=="AdversarialAutoencoder":
                    _,recon= model(sequences)
                val_loss = eval_criterion(recon, sequences)
                if val_loss.dim() > 1:
                    val_loss = val_loss
                else:
                    val_loss = val_loss.unsqueeze(dim=0)
                    labels = labels.unsqueeze(dim=0)
                val_loss = val_loss.sum(dim=1)
                all_val_losses.extend(val_loss.cpu().numpy())
                all_val_labels.extend(labels.flatten().cpu().numpy())     
        threshold_1,std_mse = compute_threshold(all_val_losses,k=1)
        all_val_losses = np.array(all_val_losses).squeeze()  
        all_val_labels = np.array(all_val_labels).squeeze()  
        # If intrusion score > threshold, predict 1 (intrusion), else 0 (benign)
        # For FDR, get True Positives (TP) and False Positives (FP)
        
        predictions = (all_val_losses > threshold_1).astype(int)
        accuracy = accuracy_score(all_val_labels, predictions)
        print(f"Val: Accuracy: {accuracy:.4f}  ")
        return threshold_1,len(all_val_losses)
    def evaluate(self, parameters, config):
        #focuses on server-side evaluation, so we can keep this simple
        return 0.0, 0, {}


In [10]:
class FedAnomalyStrategy(fl.server.strategy.FedAvg):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.aggregated_threshold: Optional[float] = None

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[fl.common.Parameters], Dict[str, Scalar]]:
        
        aggregated_parameters, _ = super().aggregate_fit(server_round, results, failures)
        candidates = [
            (res.metrics["threshold"],res.metrics["len"]) for _, res in results if "threshold" in res.metrics
        ]
        if candidates:
            threholds=np.array([pair[0] for pair in candidates],dtype=float)
            weights=np.array([pair[1] for pair in candidates],dtype=int)

            self.aggregated_threshold = np.average(threholds,weights=weights)
            print(f"Round {server_round}: Aggregated threshold = {self.aggregated_threshold:.4f}")
        else:
            print("Warning: No thresholds received from clients.")
        return aggregated_parameters, {}

In [11]:

# ==============================================================================
#  SERVER-SIDE LOGIC AND SIMULATION START
# ==============================================================================

def client_function(context:Context ) -> FlowerClient:
    client_id = int(context.node_config["partition-id"])
    trainloader = load_data_from_id(client_id,"client",chunk_size=1)
    valloader = load_data_from_id(client_id,"server",chunk_size=1)
    model = get_model().to(DEVICE)
    return FlowerClient(client_id, model, trainloader,valloader).to_client()

def make_client_fn_with_cache(chunk_size=10000):
    print("...")
    dataloader_cache: Dict[str, DataLoader] = {}
    def client_fn(context:Context ) -> FlowerClient:
        client_id = int(context.node_config["partition-id"])
        if client_id not in dataloader_cache:
            # If not, create it once and store it in the cache
            print(f"Round 1: Loading and caching data for client {client_id}...")
            dataloader_cache[client_id] = load_data_from_id(client_id,"client",chunk_size=chunk_size)
        else:
            print(f"Reusing cached dataloader for client {client_id}...")
        trainloader = dataloader_cache[client_id]
        model = get_model().to(DEVICE)
        return FlowerClient(client_id, model, trainloader).to_client()
    return client_fn


def get_evaluate_fn(model, test_dataloader, strategy: FedAnomalyStrategy):
    """Return an evaluation function for server-side evaluation with caching """
    eval_criterion = nn.MSELoss(reduction='none').to(DEVICE)
    best_f1=0
    best_recall=0
    def evaluate(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
        train_model=True
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        nonlocal best_f1,best_recall
        params_dict = zip(model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        model.load_state_dict(state_dict, strict=True)
        model.to(DEVICE)
        model.eval()
        all_test_losses = []
        all_test_labels = []
        temp_best_recall =best_recall
        temp_best_f1 =best_f1
        with torch.no_grad():
            for sequences, labels in test_dataloader:
                sequences = sequences.squeeze().to(DEVICE)
                labels = labels.squeeze().to(DEVICE)
                if model._get_name()=="AE":
                    recon = model(sequences)
                elif model._get_name()=="VAE" :
                    recon, mu, logvar = model(sequences)
                elif model._get_name()=="AdversarialAutoencoder":
                    _,recon= model(sequences)

                intrusion_scores = eval_criterion(recon, sequences)
                if intrusion_scores.dim() > 1:
                    intrusion_scores = intrusion_scores
                else:
                    intrusion_scores = intrusion_scores.unsqueeze(dim=0)
                    labels = labels.unsqueeze(dim=0)
                if intrusion_scores.dim()==3:
                    ##GRU : mean of window
                    intrusion_scores = intrusion_scores.mean(dim=1)
                intrusion_scores = intrusion_scores.sum(dim=1)
                all_test_losses.extend(intrusion_scores.cpu().numpy())
                all_test_labels.extend(labels.cpu().numpy())

        all_test_losses = np.array(all_test_losses)
        all_test_labels = np.array(all_test_labels)
        if strategy.aggregated_threshold is None:
            # Threshold not available yet (e.g., round 0)
            threshold=0
        else:
            threshold = strategy.aggregated_threshold
        test_result = {}

        predictions = (all_test_losses > threshold).astype(int)
        binary_test_labels = (all_test_labels != 0).astype(int)

        # Find the indices where the prediction was incorrect
        misclassified_indices = np.where(binary_test_labels != predictions)[0]

        # Get the original labels for those misclassified instances
        misclassified_original_labels = all_test_labels[misclassified_indices]

        # To get a summary count of which labels were misclassified
        print("Counts of : original binary labels",Counter(binary_test_labels),"predicted binary labels",Counter(predictions))
        print(f"Counts of  original  labels: {dict(sorted(Counter(all_test_labels).items()))}")
        print(f"Counts of misclassified original labels: {dict(sorted(Counter(misclassified_original_labels).items()))}")
        accuracy = accuracy_score(binary_test_labels, predictions)
        f1 = f1_score(binary_test_labels, predictions, zero_division=0)
        recall = recall_score(binary_test_labels, predictions,zero_division=0)
        _, fp, _, tp = confusion_matrix(binary_test_labels, predictions, labels=[0, 1]).ravel()
        # FDR = FP / (FP + TP) 
        if (fp + tp) == 0:
            fdr = 0.0 
        else:
            fdr = fp / (fp + tp)
        test_result[0] = f"threshold={threshold:.4f} ,Test : Accuracy: {accuracy:.4f} Recall : {recall:.4f} FDR: {fdr:.4f}  F1-score: {f1:.4f} "
        print(test_result)
        !mkdir fed_best_models -p
        if f1>best_f1 :
            best_f1=f1
        if recall>best_recall:
            best_recall=recall
        if ((best_recall>temp_best_recall or best_f1 > temp_best_f1) and not(strategy.aggregated_threshold is None)):
            if train_model:
                save_path ="fed_best_models/"+cfg.STRATEGY+"_"+model._get_name()+"_f1_"+f"{best_f1:.2f}" +"_recall_"+f"{best_recall:.2f}" +"_.pth"
                torch.save(model.state_dict(),save_path)
                print("model",model._get_name(),"is saved in" ,save_path )
        return np.sum(all_test_losses)/len(all_test_losses),test_result

    return evaluate


In [12]:

def get_initial_parameters(model_name: str):
    """
    Initializes the model weights using Xavier uniform distribution
    and returns them as a Flower Parameters object.
    """
    
    temp_model = get_model()
    for param in temp_model.parameters():
        if param.dim() > 1:
            nn.init.xavier_uniform_(param)
            
    ndarrays = [val.cpu().numpy() for _, val in temp_model.state_dict().items()]
    return ndarrays_to_parameters(ndarrays)


def load_data_from_id(id: int, node = "client" ,chunk_size=10000):
    """Loads the data for a specific training client."""
    if node == "client":
        file_list = TRAIN_CLIENT_DATA_MAPPING[id]
        shuffle=cfg.SHUFFLE_FILES
    else: # server
        file_list = SERVER_EVALUATION_DATA_MAPPING[id]
        shuffle = False
    ## means load all chunks of data in memory at once 
    if chunk_size==10000:
        train_loader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=chunk_size,
                batch_size=1 ,
                csv_files=file_list,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=64,shuffle=shuffle)
    else :
        train_loader=DataLoader(ModbusFlowStream(
                shuffle=shuffle,
                chunk_size=chunk_size,
                batch_size=cfg.BATCH_SIZE ,
                csv_files=file_list,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    return train_loader

def get_model():
    """Returns the model specified in the config."""
    if cfg.MODEL_NAME == "VAE":
        return VAE(input_dim=cfg.INPUT_DIM)
    elif cfg.MODEL_NAME == "AE":
        return AE(input_dim=cfg.INPUT_DIM)
    elif cfg.MODEL_NAME =="AAE":
        return AdversarialAutoencoder()#76
    else:
        raise ValueError(f"Unknown model name: {cfg.MODEL_NAME}. Choose 'AE' or 'VAE' or 'AAE'.")

def set_server_strategy():
    if cfg.STRATEGY == "FED_PROX":
        print(f"Using FedProx strategy with {cfg.MODEL_NAME} model.")
    else:
        print(f"Using FedAvg strategy with {cfg.MODEL_NAME} model.")
    strategy = FedAnomalyStrategy(
        fraction_fit=1.0,
        fraction_evaluate=0.0,
        min_fit_clients=cfg.NUM_TRAIN_CLIENTS,
        min_available_clients=cfg.NUM_TRAIN_CLIENTS,
        min_evaluate_clients=0,
        initial_parameters=get_initial_parameters(cfg.MODEL_NAME)
    )
    model = get_model().to(DEVICE)
    testloader = load_data_from_id(-1,"server",chunk_size=1)
    evaluate_function = get_evaluate_fn(model, testloader, strategy)
    strategy.evaluate_fn=evaluate_function
    return strategy


#### External Attack

In [13]:
[col for col in modbus.dataset["attack_dataset_dir"]["external"] if col.find("network-wide") !=-1]

['./ModbusDataset/attack/external/network-wide/ready/network-wide-normal-1-labeled.csv',
 './ModbusDataset/attack/external/network-wide/ready/network-wide-normal-0-labeled.csv']

In [13]:

# # ==============================================================================
# #  DATA Distribution
# # ==============================================================================



network_train_files=[col for col in modbus.dataset["benign_dataset_dir"] if col.find("network-wide")!=-1][:]
test_files=[[col for col in modbus.dataset["attack_dataset_dir"]["external"] if col.find("network-wide") !=-1]]

# ### (in comp scada scenario)missed attack logs files for the day 21 for ied1b which can reduce the accuracy.
# test_files.remove(dataset_directory+"attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")

random.seed(45)
random.shuffle(network_train_files)

num_splits = 4
train_files = list(np.array_split(network_train_files, num_splits))

SERVER_EVALUATION_DATA_MAPPING = [list(traffic_file[-1:]) for traffic_file in train_files ]
SERVER_EVALUATION_DATA_MAPPING.extend(test_files)
TRAIN_CLIENT_DATA_MAPPING = [list(traffic_file[:-1]) for traffic_file in train_files ]

for i in range(num_splits):
    print("node_",i+1,"train:",len(TRAIN_CLIENT_DATA_MAPPING[i]),TRAIN_CLIENT_DATA_MAPPING[i])
    print("node_",i+1,"val:",len(SERVER_EVALUATION_DATA_MAPPING[i]),SERVER_EVALUATION_DATA_MAPPING[i])

print("test",len(SERVER_EVALUATION_DATA_MAPPING[-1]),SERVER_EVALUATION_DATA_MAPPING[-1])


node_ 1 train: 4 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-23-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-17-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-20-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-29-labeled.csv']
node_ 1 val: 1 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-24-labeled.csv']
node_ 2 train: 4 ['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-14-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-26-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-21-labeled.csv', './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-32-labeled.

In [16]:

# ==============================================================================
#  CONFIGURATION: TWEAK  FEDERATED LEARNING EXPERIMENT
# ==============================================================================
class Config:
    """Global configuration class for the federated learning experiment."""
    # --- FL Parameters ---
    NUM_TRAIN_CLIENTS = num_splits
    NUM_ROUNDS = 5
    LOCAL_EPOCHS = 3
    BATCH_SIZE = 64
    LEARNING_RATE = 1e-3
    WEIGHT_DECAY = 1e-5
    
    # --- Strategy Selection ---
    # Choose from "FED_AVG", "FED_PROX"
    STRATEGY = "FED_AVG" 
    PROXIMAL_MU = 1e-2 # Proximal term for FedProx
    # --- Model Selection ---
    # Choose from "AE" (Autoencoder) or "VAE" (Variational Autoencoder) or "AdverserialAutoencoder"
    MODEL_NAME = "AE"
    INPUT_DIM = 76
    # --- Anomaly Detection ---
    SHUFFLE_FILES=  True
# Instantiate the configuration
cfg = Config()

loaded_scalers = load_scalers("fitted_scalers")
import warnings




Successfully loaded scalers for 'network-wide'


In [16]:
# ## cache all client data in memory
# client_fn=make_client_fn_with_cache()

# for i in range(3):
#     context= Context(0,i,{},0,0)
#     context.node_config["partition-id"]=i
#     client_fn(context)


In [17]:
warnings.filterwarnings("ignore", category=UserWarning, message=".*DEPRECATED FEATURE: flwr.simulation.start_simulation.*")
warnings.filterwarnings("ignore", category=DeprecationWarning)

strategy=set_server_strategy()


history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

Using FedAvg strategy with AE model.


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2025-07-26 22:03:08,112	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 1888005734.0, 'memory': 3776011470.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower

Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({1: 221329})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 156058}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}


INFO :      initial parameters (loss, other metrics): 16.916452430544574, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


(ClientAppActor pid=2271) Train : time 45.86 s Epoch 1
(ClientAppActor pid=2271) Train Loss: 2.1506
(ClientAppActor pid=2271) Train : time 47.68 s Epoch 2
(ClientAppActor pid=2271) Train Loss: 0.0437
(ClientAppActor pid=2271) Train : time 39.40 s Epoch 3
(ClientAppActor pid=2271) Train Loss: 0.0314
(ClientAppActor pid=2271) -----------mse_loss mean :  0.0335 std: 0.3323
(ClientAppActor pid=2271) Val: Accuracy: 0.9929  
(ClientAppActor pid=2271) Train : time 31.09 s Epoch 1
(ClientAppActor pid=2271) Train Loss: 1.8821
(ClientAppActor pid=2271) Train : time 28.82 s Epoch 2
(ClientAppActor pid=2271) Train Loss: 0.0571
(ClientAppActor pid=2271) Train : time 29.45 s Epoch 3
(ClientAppActor pid=2271) Train Loss: 0.0274
(ClientAppActor pid=2271) -----------mse_loss mean :  0.0260 std: 0.2837
(ClientAppActor pid=2271) Val: Accuracy: 0.9932  
(ClientAppActor pid=2271) Train : time 32.89 s Epoch 1
(ClientAppActor pid=2271) Train Loss: 1.4394
(ClientAppActor pid=2271) Train : time 33.91 s Epoch 2

...

(ClientAppActor pid=2271) Train : time 40.22 s Epoch 1
(ClientAppActor pid=2271) Train Loss: 0.0074
(ClientAppActor pid=2271) Train : time 38.92 s Epoch 2
(ClientAppActor pid=2271) Train Loss: 0.0064
(ClientAppActor pid=2271) Train : time 36.05 s Epoch 3
(ClientAppActor pid=2271) Train Loss: 0.0040
(ClientAppActor pid=2271) -----------mse_loss mean :  0.0022 std: 0.0462
(ClientAppActor pid=2271) Val: Accuracy: 0.9964  
(ClientAppActor pid=2271) Train : time 32.66 s Epoch 1
(ClientAppActor pid=2271) Train Loss: 0.0086
(ClientAppActor pid=2271) Train : time 34.81 s Epoch 2
(ClientAppActor pid=2271) Train Loss: 0.0075
(ClientAppActor pid=2271) Train : time 32.52 s Epoch 3
(ClientAppActor pid=2271) Train Loss: 0.0072
(ClientAppActor pid=2271) -----------mse_loss mean :  0.0060 std: 0.0917
(ClientAppActor pid=2271) Val: Accuracy: 0.9958  
(ClientAppActor pid=2271) Train : time 36.49 s Epoch 1
(ClientAppActor pid=2271) Train Loss: 0.0086
(ClientAppActor pid=2271) Train : time 38.11 s Epoch 2

INFO :      aggregate_fit: received 4 results and 0 failures


(ClientAppActor pid=2271) -----------mse_loss mean :  0.0075 std: 0.1102
(ClientAppActor pid=2271) Val: Accuracy: 0.9948  
Round 5: Aggregated threshold = 0.0820
Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({0: 155262, 1: 66067})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 957, 1: 161}
{0: 'threshold=0.0820 ,Test : Accuracy: 0.9949 Recall : 0.9975 FDR: 0.0145  F1-score: 0.9915 '}


INFO :      fit progress: (5, 0.6322511538253008, {0: 'threshold=0.0820 ,Test : Accuracy: 0.9949 Recall : 0.9975 FDR: 0.0145  F1-score: 0.9915 '}, 2448.011376068)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 2448.01s
INFO :      	History (loss, centralized):
INFO :      		round 0: 16.916452430544574
INFO :      		round 1: 1.245032564643585
INFO :      		round 2: 1.5683124726086504
INFO :      		round 3: 0.5364802299404958
INFO :      		round 4: 0.6462467215096078
INFO :      		round 5: 0.6322511538253008
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  '
INFO :      	      'F1-score: 0.4555 '),
INFO :      	     (1,
INFO :      	      'threshold=0.2639 ,Test : Accuracy: 0.6261 Recall : 0.8461 FDR: 0.5683  '
INFO :      	      'F1-score: 0.5717 '),
INFO :      	     (2,
INFO :   

Federated learning simulation finished.


In [17]:

# Instantiate the configuration
cfg.STRATEGY="FED_PROX"
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout


Using FedProx strategy with AE model.
Starting federated learning simulation...


2025-07-26 23:34:59,298	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 1709929267.0, 'memory': 3419858535.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({1: 221329})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 156058}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}


INFO :      initial parameters (loss, other metrics): 16.93721112009723, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

model AE is saved in fed_best_models/FED_PROX_AE_f1_0.99_recall_1.00_.pth
(ClientAppActor pid=21780) Train : time 71.13 s Epoch 1
(ClientAppActor pid=21780) Train Loss: 0.0120
(ClientAppActor pid=21780) Train : time 74.82 s Epoch 2
(ClientAppActor pid=21780) Train Loss: 0.0120
(ClientAppActor pid=21780) Train : time 74.42 s Epoch 3
(ClientAppActor pid=21780) Train Loss: 0.0119
(ClientAppActor pid=21780) -----------mse_loss mean :  0.0094 std: 0.1092
(ClientAppActor pid=21780) Val: Accuracy: 0.9944  
(ClientAppActor pid=21780) Train : time 62.52 s Epoch 1
(ClientAppActor pid=21780) Train Loss: 0.0124
(ClientAppActor pid=21780) Train : time 61.94 s Epoch 2
(ClientAppActor pid=21780) Train Loss: 0.0122
(ClientAppActor pid=21780) Train : time 61.64 s Epoch 3
(ClientAppActor pid=21780) Train Loss: 0.0123
(ClientAppActor pid=21780) -----------mse_loss mean :  0.0134 std: 0.1076
(ClientAppActor pid=21780) Val: Accuracy: 0.9953  
(ClientAppActor pid=21780) Train : time 53.17 s Epoch 1
(ClientA

INFO :      aggregate_fit: received 4 results and 0 failures


(ClientAppActor pid=21780) -----------mse_loss mean :  0.0132 std: 0.1290
Round 5: Aggregated threshold = 0.1373
(ClientAppActor pid=21780) Val: Accuracy: 0.9928  
Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({0: 155254, 1: 66075})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 957, 1: 153}
{0: 'threshold=0.1373 ,Test : Accuracy: 0.9950 Recall : 0.9977 FDR: 0.0145  F1-score: 0.9915 '}


INFO :      fit progress: (5, 0.7993914743436242, {0: 'threshold=0.1373 ,Test : Accuracy: 0.9950 Recall : 0.9977 FDR: 0.0145  F1-score: 0.9915 '}, 4487.011939446001)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 4487.01s
INFO :      	History (loss, centralized):
INFO :      		round 0: 16.93721112009723
INFO :      		round 1: 0.6500179032119605
INFO :      		round 2: 0.4974349201302134
INFO :      		round 3: 0.6186465482381432
INFO :      		round 4: 0.7948349798264123
INFO :      		round 5: 0.7993914743436242
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  '
INFO :      	      'F1-score: 0.4555 '),
INFO :      	     (1,
INFO :      	      'threshold=0.3713 ,Test : Accuracy: 0.8997 Recall : 0.9977 FDR: 0.2529  '
INFO :      	      'F1-score: 0.8544 '),
INFO :      	     (2,
INFO :

Federated learning simulation finished.


### VAE

In [18]:
# Instantiate the configuration
cfg.STRATEGY="FED_AVG"
cfg.MODEL_NAME="VAE"
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 4},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout


Using FedAvg strategy with VAE model.
Starting federated learning simulation...


2025-07-27 00:44:18,687	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 3333827790.0, 'object_store_memory': 1666913894.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({1: 221329})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 156058}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}


INFO :      initial parameters (loss, other metrics): 18.036986793416137, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

Round 5: Aggregated threshold = 0.4054
(ClientAppActor pid=36318) -----------mse_loss mean :  0.0498 std: 0.3166
(ClientAppActor pid=36318) Val: Accuracy: 0.9892  
Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({0: 167338, 1: 53991})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 1235, 1: 12507, 6: 8}
{0: 'threshold=0.4054 ,Test : Accuracy: 0.9379 Recall : 0.8083 FDR: 0.0229  F1-score: 0.8847 '}


INFO :      fit progress: (5, 0.5403309383316239, {0: 'threshold=0.4054 ,Test : Accuracy: 0.9379 Recall : 0.8083 FDR: 0.0229  F1-score: 0.8847 '}, 2531.6515583889995)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 2531.65s
INFO :      	History (loss, centralized):
INFO :      		round 0: 18.036986793416137
INFO :      		round 1: 0.9953139613426166
INFO :      		round 2: 0.5350337478369306
INFO :      		round 3: 0.5416102837856765
INFO :      		round 4: 0.5357206494630166
INFO :      		round 5: 0.5403309383316239
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  '
INFO :      	      'F1-score: 0.4555 '),
INFO :      	     (1,
INFO :      	      'threshold=0.4155 ,Test : Accuracy: 0.8480 Recall : 0.9977 FDR: 0.3396  '


model VAE is saved in fed_best_models/FED_AVG_VAE_f1_0.88_recall_1.00_.pth


INFO :      	      'F1-score: 0.7947 '),
INFO :      	     (2,
INFO :      	      'threshold=0.5662 ,Test : Accuracy: 0.8177 Recall : 0.4019 FDR: 0.0475  '
INFO :      	      'F1-score: 0.5653 '),
INFO :      	     (3,
INFO :      	      'threshold=0.5655 ,Test : Accuracy: 0.8190 Recall : 0.4032 FDR: 0.0405  '
INFO :      	      'F1-score: 0.5678 '),
INFO :      	     (4,
INFO :      	      'threshold=0.5436 ,Test : Accuracy: 0.8188 Recall : 0.4026 FDR: 0.0403  '
INFO :      	      'F1-score: 0.5673 '),
INFO :      	     (5,
INFO :      	      'threshold=0.4054 ,Test : Accuracy: 0.9379 Recall : 0.8083 FDR: 0.0229  '
INFO :      	      'F1-score: 0.8847 ')]}
INFO :      


Federated learning simulation finished.


In [19]:
cfg.STRATEGY="FED_PROX"
strategy=set_server_strategy()

print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 4},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout


Using FedProx strategy with VAE model.
Starting federated learning simulation...


2025-07-27 01:23:28,377	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 3277833831.0, 'object_store_memory': 1638916915.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({1: 221329})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 156058}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}


INFO :      initial parameters (loss, other metrics): 17.708340298831153, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

Round 5: Aggregated threshold = 0.4608
(ClientAppActor pid=44427) -----------mse_loss mean :  0.0694 std: 0.4085
(ClientAppActor pid=44427) Val: Accuracy: 0.9902  
Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({0: 169831, 1: 51498})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 1457, 1: 15222, 6: 8}
{0: 'threshold=0.4608 ,Test : Accuracy: 0.9246 Recall : 0.7667 FDR: 0.0283  F1-score: 0.8571 '}


INFO :      fit progress: (5, 0.5313830386551243, {0: 'threshold=0.4608 ,Test : Accuracy: 0.9246 Recall : 0.7667 FDR: 0.0283  F1-score: 0.8571 '}, 3821.811365921001)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 3821.81s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.708340298831153
INFO :      		round 1: 0.5551878863027438
INFO :      		round 2: 0.5316846611831256
INFO :      		round 3: 0.5308336585467788
INFO :      		round 4: 0.532405449014815
INFO :      		round 5: 0.5313830386551243
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  '
INFO :      	      'F1-score: 0.4555 '),
INFO :      	     (1,
INFO :      	      'threshold=0.5552 ,Test : Accuracy: 0.8330 Recall : 0.4774 FDR: 0.0836  '


model VAE is saved in fed_best_models/FED_PROX_VAE_f1_0.86_recall_1.00_.pth


INFO :      	      'F1-score: 0.6277 '),
INFO :      	     (2,
INFO :      	      'threshold=0.5347 ,Test : Accuracy: 0.8229 Recall : 0.4211 FDR: 0.0488  '
INFO :      	      'F1-score: 0.5837 '),
INFO :      	     (3,
INFO :      	      'threshold=0.5038 ,Test : Accuracy: 0.8985 Recall : 0.6762 FDR: 0.0290  '
INFO :      	      'F1-score: 0.7972 '),
INFO :      	     (4,
INFO :      	      'threshold=0.4844 ,Test : Accuracy: 0.9155 Recall : 0.7371 FDR: 0.0309  '
INFO :      	      'F1-score: 0.8373 '),
INFO :      	     (5,
INFO :      	      'threshold=0.4608 ,Test : Accuracy: 0.9246 Recall : 0.7667 FDR: 0.0283  '
INFO :      	      'F1-score: 0.8571 ')]}
INFO :      


Federated learning simulation finished.


In [20]:
# Instantiate the configuration
cfg.STRATEGY="FED_AVG"
cfg.MODEL_NAME="AAE"
cfg.LEARNING_RATE=1e-4
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout


Using FedAvg strategy with AAE model.
Starting federated learning simulation...


2025-07-27 02:22:29,737	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 3242832692.0, 'object_store_memory': 1621416345.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({1: 221329})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 156058}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}


INFO :      initial parameters (loss, other metrics): 17.18490911719657, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

model AdversarialAutoencoder is saved in fed_best_models/FED_AVG_AdversarialAutoencoder_f1_0.99_recall_1.00_.pth
(ClientAppActor pid=56481) Train : time 65.79 s Epoch 1
(ClientAppActor pid=56481) Generator Loss: 1.1213 Discriminator Loss: 13.2397
(ClientAppActor pid=56481) Train : time 69.05 s Epoch 2
(ClientAppActor pid=56481) Generator Loss: 1.0951 Discriminator Loss: 15.4672
(ClientAppActor pid=56481) Train : time 65.02 s Epoch 3
(ClientAppActor pid=56481) Generator Loss: 1.1059 Discriminator Loss: 13.5196
(ClientAppActor pid=56481) -----------mse_loss mean :  0.0138 std: 0.2017
(ClientAppActor pid=56481) Val: Accuracy: 0.9954  
(ClientAppActor pid=56481) Train : time 49.37 s Epoch 1
(ClientAppActor pid=56481) Generator Loss: 1.0964 Discriminator Loss: 13.4367
(ClientAppActor pid=56481) Train : time 51.22 s Epoch 2
(ClientAppActor pid=56481) Generator Loss: 1.0596 Discriminator Loss: 15.9394
(ClientAppActor pid=56481) Train : time 48.89 s Epoch 3
(ClientAppActor pid=56481) Generator

INFO :      aggregate_fit: received 4 results and 0 failures


Round 5: Aggregated threshold = 0.2121
(ClientAppActor pid=56481) -----------mse_loss mean :  0.0130 std: 0.1916
(ClientAppActor pid=56481) Val: Accuracy: 0.9962  
Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({0: 155294, 1: 66035})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 929, 1: 165}
{0: 'threshold=0.2121 ,Test : Accuracy: 0.9951 Recall : 0.9975 FDR: 0.0141  F1-score: 0.9917 '}


INFO :      fit progress: (5, 1.2696438333883042, {0: 'threshold=0.2121 ,Test : Accuracy: 0.9951 Recall : 0.9975 FDR: 0.0141  F1-score: 0.9917 '}, 4013.688750272002)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 4013.69s
INFO :      	History (loss, centralized):
INFO :      		round 0: 17.18490911719657
INFO :      		round 1: 0.5044641193088117
INFO :      		round 2: 0.5094359669315814
INFO :      		round 3: 1.402052346280876
INFO :      		round 4: 1.2725523993692647
INFO :      		round 5: 1.2696438333883042
INFO :      	History (metrics, centralized):
INFO :      	{0: [(0,
INFO :      	      'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  '
INFO :      	      'F1-score: 0.4555 '),
INFO :      	     (1,
INFO :      	      'threshold=0.3635 ,Test : Accuracy: 0.9901 Recall : 0.9974 FDR: 0.0302  '
INFO :      	      'F1-score: 0.9834 '),
INFO :      	     (2,
INFO : 

Federated learning simulation finished.


In [23]:
# Instantiate the configuration
cfg.STRATEGY="FED_PROX"
cfg.MODEL_NAME="AAE"
cfg.LEARNING_RATE=1e-4
strategy=set_server_strategy()

# --- Start the Simulation ---
print("Starting federated learning simulation...")
history = fl.simulation.start_simulation(
    client_fn=client_function,
    num_clients=cfg.NUM_TRAIN_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=cfg.NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1} if DEVICE.type == "cuda" else {"num_cpus": 1},
)
print("Federated learning simulation finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout


Using FedProx strategy with AAE model.
Starting federated learning simulation...


2025-07-27 09:25:23,522	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'memory': 3270986958.0, 'object_store_memory': 1635493478.0, 'GPU': 1.0, 'node:172.24.78.91': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({1: 221329})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 156058}
{0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}


INFO :      initial parameters (loss, other metrics): 17.891992237799837, {0: 'threshold=0.0000 ,Test : Accuracy: 0.2949 Recall : 1.0000 FDR: 0.7051  F1-score: 0.4555 '}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


...

(ClientAppActor pid=139326) -----------mse_loss mean :  0.0471 std: 0.2836
Round 1: Aggregated threshold = 0.2762
(ClientAppActor pid=139326) Val: Accuracy: 0.9932  
Counts of : original binary labels Counter({0: 156058, 1: 65271}) predicted binary labels Counter({0: 155275, 1: 66054})
Counts of  original  labels: {0: 156058, 1: 65233, 2: 1, 3: 1, 4: 1, 5: 2, 6: 32, 7: 1}
Counts of misclassified original labels: {0: 951, 1: 168}
{0: 'threshold=0.2762 ,Test : Accuracy: 0.9949 Recall : 0.9974 FDR: 0.0144  F1-score: 0.9915 '}


INFO :      fit progress: (1, 0.48734678493328937, {0: 'threshold=0.2762 ,Test : Accuracy: 0.9949 Recall : 0.9974 FDR: 0.0144  F1-score: 0.9915 '}, 1304.0023682550018)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 4 clients (out of 4)


model AdversarialAutoencoder is saved in fed_best_models/FED_PROX_AdversarialAutoencoder_f1_0.99_recall_1.00_.pth
(ClientAppActor pid=139326) Train : time 84.67 s Epoch 1
(ClientAppActor pid=139326) Generator Loss: 4.1141 Discriminator Loss: 0.9664
(ClientAppActor pid=139326) Train : time 88.04 s Epoch 2
(ClientAppActor pid=139326) Generator Loss: 4.0689 Discriminator Loss: 1.5014
(ClientAppActor pid=139326) Train : time 95.11 s Epoch 3
(ClientAppActor pid=139326) Generator Loss: 3.5488 Discriminator Loss: 2.5702
(ClientAppActor pid=139326) -----------mse_loss mean :  0.0196 std: 0.1664
(ClientAppActor pid=139326) Val: Accuracy: 0.9931  
(ClientAppActor pid=139326) Train : time 119.94 s Epoch 1
(ClientAppActor pid=139326) Generator Loss: 4.1929 Discriminator Loss: 1.0361
(ClientAppActor pid=139326) Train : time 119.88 s Epoch 2
(ClientAppActor pid=139326) Generator Loss: 4.1083 Discriminator Loss: 2.3250


KeyboardInterrupt: 

(ClientAppActor pid=139326) Train : time 119.33 s Epoch 3
(ClientAppActor pid=139326) Generator Loss: 3.0342 Discriminator Loss: 2.9670
(ClientAppActor pid=139326) -----------mse_loss mean :  0.0159 std: 0.1333
(ClientAppActor pid=139326) Val: Accuracy: 0.9944  
(ClientAppActor pid=139326) Train : time 119.72 s Epoch 1
(ClientAppActor pid=139326) Generator Loss: 4.1789 Discriminator Loss: 1.0082
(ClientAppActor pid=139326) Train : time 108.03 s Epoch 2
(ClientAppActor pid=139326) Generator Loss: 4.0193 Discriminator Loss: 2.0101
(ClientAppActor pid=139326) Train : time 113.99 s Epoch 3
(ClientAppActor pid=139326) Generator Loss: 2.9522 Discriminator Loss: 2.7669
(ClientAppActor pid=139326) -----------mse_loss mean :  0.0166 std: 0.1413
(ClientAppActor pid=139326) Val: Accuracy: 0.9928  
(ClientAppActor pid=139326) Train : time 106.53 s Epoch 1
(ClientAppActor pid=139326) Generator Loss: 4.3807 Discriminator Loss: 0.9481
(ClientAppActor pid=139326) Train : time 99.86 s Epoch 2
(ClientAp

#### FedAVG evaluation

In [62]:
[col for sublist in SERVER_EVALUATION_DATA_MAPPING[:-1] for col in sublist]


['./ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-24-labeled.csv',
 './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-28-labeled.csv',
 './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-30-labeled.csv',
 './ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-22-labeled.csv']

In [69]:
dataset_directory = "./ModbusDataset" 

modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

Trained_AE_model=AE(input_dim=76)
Trained_AE_model.load_state_dict(torch.load("./fed_best_models/FED_AVG_AE_f1_0.99_recall_1.00_.pth"))
Trained_VAE_model=VAE(input_dim=76)
Trained_VAE_model.load_state_dict(torch.load("./fed_best_models/FED_AVG_VAE_f1_0.88_recall_1.00_.pth"))
Trained_AAE_model=AdversarialAutoencoder()
Trained_AAE_model.load_state_dict(torch.load("./fed_best_models/FED_AVG_AdversarialAutoencoder_f1_0.99_recall_1.00_.pth"))
val_files=[col for sublist in SERVER_EVALUATION_DATA_MAPPING[:-1] for col in sublist]


for scenario in {"compromised-ied","external","compromised-scada"}:
    if scenario=="compromised-scada":
        print("scenario :",scenario)
        dataset_directory = "./ModbusDataset" 
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("ied1b")!=-1]
        ### missed attack logs for the day 21 for ied1b which can reduce the accuracy.
        test_files.remove(dataset_directory+"/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv")    

    elif scenario=="compromised-ied":
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("trust-scada-hmi")!=-1]
    else:
        print("scenario :",scenario)
        test_files= [col for col in modbus.dataset["attack_dataset_dir"][scenario] if col.find("network-wide")!=-1]        

    print("----------- benign valid files:",len(val_files),val_files)
    print(f"----------{scenario} attack  test files : ",len(test_files),test_files)
    val_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=val_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    test_dataloader=DataLoader(ModbusFlowStream(
                shuffle=False,
                chunk_size=1,
                batch_size=64,
                csv_files=test_files,
                scalers=loaded_scalers['network-wide']['min_max_scalers'],
            ),batch_size=1,shuffle=False)
    for trained_model in {Trained_AE_model,Trained_VAE_model,Trained_AAE_model}:
        print("*"*10,trained_model._get_name(),10*"*")
        train_eval(trained_model,None,val_dataloader,test_dataloader,shuffle_files=False,num_epochs=1,eval_epoch=1,criterion_method="mse",train_model=False,learning_rates=[0],weight_decays=[0])
        

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and